In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

from pyspark.sql import SQLContext
from pyspark import SparkConf, SparkContext
ctx = SQLContext(SparkContext(conf = (SparkConf().setMaster('local[*]').setAppName('quoteExtraction'))))

In [2]:
from settings import *
from utils import *
from quoteExtraction import *

In [3]:
def quoteExtraction(limitDocuments=10):
    query = createQuery(limitDocuments, 'web')
    documents = queryDB(query)        

    if useSpark:
        rddd = ctx.createDataFrame(documents[['title','body']]).rdd
        documents['quotes'] = rddd.map(lambda s: dependencyGraphSearch(s.title, s.body)).collect()
    else:
        documents['quotes'] = documents.apply(lambda d: dependencyGraphSearch(d['title'],d['body']), axis=1)
    
    documents = documents[['topic_label']].join(documents['quotes'].apply(pd.Series).stack().reset_index(level=1, drop=True).apply(pd.Series))
    
    print ('#quotesPerDocument: ',len(documents)/limitDocuments)
    return documents

if useCache:
    documents = pd.read_pickle(cachedDataFrame)
else:
    documents = quoteExtraction(20)
    documents.to_pickle(cachedDataFrame)

#quotesPerDocument:  5.8


In [4]:
documents[['topic_label', 'quotee']].groupby('topic_label').size()

topic_label
Addiction                               3
Bacteria, Disease                       8
Consumer, Demand, Supply                4
Farm, Agriculture, Farmer              11
Fresh Food, Real Food, Healthy Food     1
Junk Food, Unhealthy Food              11
Law, Label, Labeling, GMO              15
Obesity, Fat, Weight                   18
Research, Advancement, Progress         8
Salt, Sugar, Fat, Recommendation       14
Sugar, Diabetes                        10
Water Shortage                         13
dtype: int64